In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/kirell/work/emv/notebook


In [2]:
import random
import textwrap

# LOCAL imports
import emv
import emv.utils
import emv.io.media
import emv.features.text
import emv.llm.vault
import emv.db.queries


from emv.db.dao import DataAccessObject
DataAccessObject().set_user_id(3)

LOG = emv.utils.get_logger()

/home/kirell/.cache/pypoetry/virtualenvs/emv-IgKv7S_O-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/kirell/.cache/pypoetry/virtualenvs/emv-IgKv7S_O-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_params" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/kirell/.cache/pypoetry/virtualenvs/emv-IgKv7S_O-py3.10/lib/python3.10/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [ ]:
# llm = emv.llm.query.LLMWrapper() 
retriever = emv.llm.retriever.DocumentRetriever()

In [4]:
llb = emv.llm.ingest.get_vault_llm_backend()

In [5]:
query_str = "Suisse, EPFL, recherche"

In [8]:
from llama_index.core.response.notebook_utils import display_source_node

retrieved_nodes = llb.retrieve(query_str)
for node in retrieved_nodes:
    display_source_node(node, source_length=1000, show_source_metadata=True)

**Node ID:** 186<br>**Similarity:** 0.4140431317356654<br>**Text:** Au niveau du financement, j'ai d'abord demandé aux banques et aussi à la fondation des entreprises.<br>**Metadata:** {'media_id': 'rts-ZU021020-M010', 'feature_id': 186}<br>

**Node ID:** 1022<br>**Similarity:** 0.4135708224912372<br>**Text:** Ce serait un changement de paradigme. Les transports publics fonctionnent très bien en Suisse, aussi car tout est organisé par une seule entité qui coordonne tout.<br>**Metadata:** {'media_id': 'rts-ZU089021-M002', 'feature_id': 1022}<br>

**Node ID:** 1021<br>**Similarity:** 0.4090492215312682<br>**Text:** Officiellement, ils ne veulent pas encore prendre position sur le fond. Mais selon eux, abandonner le monopole comporte des risques.<br>**Metadata:** {'media_id': 'rts-ZU089021-M001', 'feature_id': 1021}<br>

**Node ID:** 891<br>**Similarity:** 0.4087009413076752<br>**Text:** L'année dernière, chez Tagoyer, nous avons racheté dans le marché. Donc nous avons fait des opérations de rachat, une vingtaine de millions de francs suisses mondialement.<br>**Metadata:** {'media_id': 'rts-ZU065016-M003', 'feature_id': 891}<br>

**Node ID:** 164<br>**Similarity:** 0.408539650781264<br>**Text:** Entre ses mains, Nicolas Dizerand manipule de l'or, ou presque. La valeur de certaines carpes koi peut atteindre plusieurs milliers, voire dizaines de milliers de francs. Des animaux d'ordement qui nécessitent des soins.<br>**Metadata:** {'media_id': 'rts-ZU018015-M000', 'feature_id': 164}<br>

In [9]:
eng = indexer.load_index()

In [14]:
indexer.query('Suisse EPFL hello')

VectorStoreQueryResult(nodes=[TextNode(id_='1022', embedding=None, metadata={'media_id': 'rts-ZU089021-M002', 'feature_id': 1022}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Ce serait un changement de paradigme. Les transports publics fonctionnent très bien en Suisse, aussi car tout est organisé par une seule entité qui coordonne tout.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), TextNode(id_='177', embedding=None, metadata={'media_id': 'rts-ZU021020-M001', 'feature_id': 177}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Alors, j'aimerais une salade de foie de volaille pour Janine, avec pas trop de salade.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), TextNode(id_='186', embedding=None, metadata=

In [10]:
response = eng.query("Suisse")
response

INFO:httpx:HTTP Request: POST http://192.168.1.42:11434/api/chat "HTTP/1.1 200 OK"


Response(response=' Switzerland is a country known for its strong economy and financial institutions. It is home to several banks and foundations that provide funding for various projects and initiatives. Additionally, the country is renowned for its production and trade of precious metals such as gold, with values reaching thousands or even tens of thousands of francs for certain items.', source_nodes=[NodeWithScore(node=TextNode(id_='186', embedding=None, metadata={'media_id': 'rts-ZU021020-M010', 'feature_id': 186}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Au niveau du financement, j'ai d'abord demandé aux banques et aussi à la fondation des entreprises.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.4472065646757102), NodeWithScore(node=TextNode(id_='164', embedding=None, metadata={'media_id': 'rts-ZU018015-M000', 'feature_id':

In [ ]:
llm = dspy.OllamaLocal("mistral", 
                       base_url="http://192.168.1.42:11434",
                       model_type='instruct')
dspy.settings.configure(lm=llm)

In [ ]:
class QueryExpanderSubQuestion(dspy.Signature):
    """Expand a query by asking sub questions about the original query in French"""
    query = dspy.InputField(desc="A query")
    expanded = dspy.OutputField(desc="expanded query with at least 500 tokens")


class QueryExpanderMoreContext(dspy.Signature):
    """Expand a query by generating more context and information about the original query in French"""
    query = dspy.InputField(desc="A query")
    expanded = dspy.OutputField(desc="expanded query with at least 500 tokens")

In [ ]:
sentence = "La Stratégie Energétique 2050 en Suisse"
sub = dspy.ChainOfThought(QueryExpanderSubQuestion)
more = dspy.ChainOfThought(QueryExpanderMoreContext)
res1 = sub(query=sentence)
res2 = more(query=sentence)

In [ ]:
ret1 = retriever.fetch_similar(res1.expanded)

In [ ]:
ret2 = retriever.fetch_similar(res2.expanded)

In [ ]:
llm.inspect_history()

In [ ]:
emv.llm.retriever.show_transcripts_from_results(ret2)

In [4]:
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

from emv.settings import DB_HOST, DB_PORT, LLM_DB_NAME, DB_USER, DB_PASSWORD, DATABASE_URL

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

Settings.embed_model = HuggingFaceEmbedding(
    model_name="camembert/camembert-large"
)

Settings.llm = llm = Ollama(base_url="http://192.168.1.42:11434", model="mistral", request_timeout=30.0)

vector_store = PGVectorStore.from_params(
    database=LLM_DB_NAME,
    host=DB_HOST,
    password=DB_PASSWORD,
    port=DB_PORT,
    user=DB_USER,
    table_name="llama_index_vector_store",
    embed_dim=1024
)

# index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
# query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("La stratégie énergétique 2050 en Suisse")
print(textwrap.fill(str(response), 100))

In [5]:
vector_store.add()

llama_index.core.indices.vector_store.base.VectorStoreIndex

In [ ]:
import emv.llm.ingest

# emv.llm.ingest.process_transcripts()

In [ ]:
emv.llm.ingest.make_db()

In [ ]:
emv.db.queries.count_features_by_type('transcript+ner')

In [ ]:
import litellm
from litellm import completion

litellm.set_verbose=False

response = completion(
            model="ollama/mixtral:8x7b-instruct-v0.1-fp16", 
            messages = [{ "content": "Tell me about Kirell Benzi","role": "user"}], 
            api_base="http://192.168.1.42:11434"
)

response.choices[0].message.content

In [ ]:
from llama_index.llms import Ollama
from llama_index.llms import ChatMessage
import textwrap

In [ ]:
llm = Ollama(base_url="http://192.168.1.42:11434", model="mistral", request_timeout=30.0)

In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a developer with a colorful personality"
    ),
    ChatMessage(role="user", content="What is my name"),
]
resp = llm.stream_chat(messages)

for r in resp:
    print(r.delta, end="")

In [ ]:
from sqlalchemy import make_url
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv()  # Load environment variables from .env file

# Get the environment variables
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

# Construct the connection string
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
url = make_url(connection_string)

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="feature",
    embed_dim=1024,  # openai embedding dimension
)

# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, show_progress=True
# )
# query_engine = index.as_query_engine()

In [ ]:
# from llama_index.core.bridge.pydantic import PrivateAttr
# from llama_index.core.embeddings import BaseEmbedding

# class LlamaIndexEmbeddings(BaseEmbedding):
#     _model: emv.features.text.TextEmbedder = PrivateAttr()
#     def __init__(
#         self,
#         **kwargs: Any,
#     ) -> None:
#         self._model = emv.features.text.TextEmbedder()
#         super().__init__(**kwargs)

#     @classmethod
#     def class_name(cls) -> str:
#         return "LlamaIndexEmbeddings"

#     async def _aget_query_embedding(self, query: str) -> List[float]:
#         return self._get_query_embedding(query)

#     async def _aget_text_embedding(self, text: str) -> List[float]:
#         return self._get_text_embedding(text)

#     def _get_query_embedding(self, query: str) -> List[float]:
#         embeddings = self._model.encode(query)
#         return embeddings[0]

#     def _get_text_embedding(self, text: str) -> List[float]:
#         embeddings = self._model.encode(text)
#         return embeddings[0]

#     def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
#         embeddings = self._model.encode(texts)
#         return embeddings